In [1]:
# حذف بسته‌های مشکل‌ساز
!pip uninstall -y numpy pandas torch transformers datasets hazm tokenizers accelerate

# نصب نسخه‌هایی که با Colab و transformers سازگار و پایدار باشند
%pip install numpy==1.26.2 pandas==2.2.2
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
%pip install datasets==2.14.0 tokenizers==0.15.2
%pip install transformers==4.41.0
%pip install git+https://github.com/sobhe/hazm.git
%pip install accelerate

Found existing installation: numpy 1.26.2
Uninstalling numpy-1.26.2:
  Successfully uninstalled numpy-1.26.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: torch 2.7.1+cpu
Uninstalling torch-2.7.1+cpu:
  Successfully uninstalled torch-2.7.1+cpu
Found existing installation: transformers 4.41.0
Uninstalling transformers-4.41.0:
  Successfully uninstalled transformers-4.41.0
Found existing installation: datasets 2.14.0
Uninstalling datasets-2.14.0:
  Successfully uninstalled datasets-2.14.0
Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1
Found existing installation: accelerate 1.9.0
Uninstalling accelerate-1.9.0:
  Successfully uninstalled accelerate-1.9.0


                                              0.0/15.8 MB ? eta -:--:--
                                              0.1/15.8 MB 2.6 MB/s eta 0:00:06
                                              0.2/15.8 MB 2.1 MB/s eta 0:00:08
     -                                        0.4/15.8 MB 3.4 MB/s eta 0:00:05
     -                                        0.7/15.8 MB 3.8 MB/s eta 0:00:05
     -                                        0.8/15.8 MB 4.1 MB/s eta 0:00:04
     --                                       0.8/15.8 MB 3.2 MB/s eta 0:00:05
     ---                                      1.2/15.8 MB 3.8 MB/s eta 0:00:04
     ---                                      1.5/15.8 MB 4.2 MB/s eta 0:00:04
     ----                                     1.9/15.8 MB 4.6 MB/s eta 0:00:04
     -----                                    2.0/15.8 MB 4.5 MB/s eta 0:00:04
     -----                                    2.2/15.8 MB 4.3 MB/s eta 0:00:04
     ------                                   2.4/15.8 MB 4

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cpu
  Obtaining dependency information for torch from https://download.pytorch.org/whl/cpu/torch-2.7.1%2Bcpu-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision from https://download.pytorch.org/whl/cpu/torchvision-0.22.1%2Bcpu-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchaudio from https://download.pytorch.org/whl/cpu/torchaudio-2.7.1%2Bcpu-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for filelock from https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for sympy>=1.13.3 from https://download.pytorch.org/whl/sympy-1.13.3-py3-none-any.whl.metadata
  Obtaining dependency information for networkx from https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata
  Obtaining dependency information for jinja2 from https://download.pytorch.org/whl/Jinja2-3.1.4-py3-none-any.whl.met

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/sobhe/hazm.git to c:\users\alireza\appdata\local\temp\pip-req-build-vzbkt3w4
  Resolved https://github.com/sobhe/hazm.git to commit d97a5ba13a680a29f2ae5bc6f862e941329890fb
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
                                              0.0/232.4 kB ? eta -:--:--
     ----------                              61.4/232.4 kB 3.2 MB/s eta 0:00:01
     --------------------                   122.9/232.4 kB 1.8 MB/s eta 0:00:01
     ------------------------------------   225.3/232.4 kB 2.0 MB/s eta 0:00:01
     ------------------------------------   225.3/232.4 kB 2.0 MB/s eta 0:00:01
     ------------------------------------   225.3/232.4 kB 2.0 MB/s eta 0:0

  Running command git clone --filter=blob:none --quiet https://github.com/sobhe/hazm.git 'C:\Users\Alireza\AppData\Local\Temp\pip-req-build-vzbkt3w4'
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/367.1 kB ? eta -:--:--
     ------                                   61.4/367.1 kB ? eta -:--:--
     ------------                           122.9/367.1 kB 1.8 MB/s eta 0:00:01
     ----------------------------------     337.9/367.1 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 367.1/367.1 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
try:
    import numpy as np
    import pandas as pd
    import torch
    from hazm import Normalizer
    from datasets import Dataset
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

    print("✅ کتابخانه‌ها با موفقیت import شدند!")
    print(f"🔹 numpy: {np.__version__}")
    print(f"🔹 pandas: {pd.__version__}")
    print(f"🔹 torch: {torch.__version__}")
    print(f"🔹 transformers: {transformers.__version__}")

except Exception as e:
    print("❌ خطا در import:", e)


c:\Users\Alireza\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


❌ خطا در import: module 'pyarrow' has no attribute 'PyExtensionType'


In [ ]:
import pandas as pd

# لینک‌ها
url1 = "https://raw.githubusercontent.com/davardoust/PHICAD/main/PHICAD-part1.csv"
url2 = "https://raw.githubusercontent.com/davardoust/PHICAD/main/PHICAD-part2.csv"

# 📥 خواندن فایل‌ها با تنظیمات دقیق
df1 = pd.read_csv(url1, sep="\t", header=0)
df2 = pd.read_csv(url2, sep="\t", header=0)

# 🔗 ادغام
df = pd.concat([df1, df2], ignore_index=True)

# حذف ستون‌های اضافه‌ای که اشتباه وارد شدن
expected_columns = ['comment_normalized', 'hate', 'spam', 'obscene', 'class']
df = df[expected_columns]

# نمایش اولیه
print(f"🔢 تعداد کل داده‌ها: {len(df)}")
print("🧾 ستون‌ها:", df.columns.tolist())
print(df.head(3))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

# حذف ردیف‌هایی که مقدار کلاس ندارن
df = df.dropna(subset=["class"])

# دوباره لیبل‌ها رو بساز
label_encoder = LabelEncoder()
df.loc[:, "label"] = label_encoder.fit_transform(df["class"])

# نمایش کلاس‌ها
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("🔢 کلاس‌ها:", label_map)

# تبدیل به Dataset
dataset = Dataset.from_pandas(df[["comment_normalized", "label"]])
print("✅ Dataset آماده است!")

In [ ]:
from hazm import Normalizer
from transformers import AutoTokenizer

normalizer = Normalizer()
model_name = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)س

def preprocess_data(texts):
    return tokenizer([normalizer.normalize(t) for t in texts], truncation=True, padding=True)

print("✅ توکنایزر و نرمالایزر آماده است!")


In [ ]:

encodings = preprocess_data(df['text'].tolist())
labels = df['label'].tolist()

dataset = Dataset.from_dict({
    'input_ids': encodings['input_ids'],
    'attention_mask': encodings['attention_mask'],
    'labels': labels
})

splits = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = splits['train']
eval_dataset = splits['test']

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./phicad_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_steps=100,
    logging_dir='./phicad_logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
trainer.train()

In [ ]:
metrics = trainer.evaluate()
print(metrics)

model.save_pretrained("phicad_model")
tokenizer.save_pretrained("phicad_model")

In [ ]:
def is_inappropriate(text):
    t = normalizer.normalize(text)
    inputs = tokenizer(t, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits
    return torch.argmax(logits, dim=1).item() == 1

comments = [
    "لعنت به این وضعیت خرابات!",
    "خیلی ممنون از خدمات شما"
]
for c in comments:
    print(c, "→", "نامناسب" if is_inappropriate(c) else "مناسب")